In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string

ModuleNotFoundError: No module named 'nltk'

In [ ]:

df_clean = pd.read_csv('documents/2019-05/Recommender System/df_clean without index.csv')
n_users = df_clean.userID.unique().shape[0]
n_items = df_clean.itemID.unique().shape[0]
count = df_clean.groupby("itemID", as_index=False).mean()
count_users = df_clean.groupby("userID", as_index=False).count()
count_users.loc[count_users['userID'] == '0013000000Eqrg2AAB']
items_df = count[['itemID']]
users_df = count_users[['userID']]
users_list = users_df.values
from sklearn.model_selection import train_test_split
trainSet, testSet = train_test_split(df_clean, test_size=0.2,stratify=df_clean['userID'])
mean_items = trainSet.groupby(['itemID']).mean()
mean_users = trainSet.groupby(['userID']).mean()
mean_overall = trainSet["rating"].mean() 
train_data_matrix_na = trainSet.pivot_table(index='itemID', columns='userID', values='rating').fillna(0)
train_data_matrix = train_data_matrix_na
train_data_matrix = train_data_matrix.loc[:, (train_data_matrix != 0).any(axis=0)]
train_data_matrix = train_data_matrix.loc[(train_data_matrix != 0).any(axis=1), :]
from scipy.spatial.distance import correlation
neighbor = NearestNeighbors(n_neighbors=4, algorithm='auto', metric = 'cosine').fit(train_data_matrix)
distances, indices = neighbor.kneighbors(train_data_matrix)


reference_index = train_data_matrix.index.values
reference_index

In [3]:
df_clean = pd.read_csv('documents/2019-05/Recommender System/df_clean without index.csv')
pivot_categorized = df_clean.pivot_table(index='itemID', columns='userID', values='rating').fillna(0)
pivot_categorized = pivot_categorized.loc[:, (pivot_categorized != 0).any(axis=0)]
pivot_categorized = pivot_categorized.loc[(pivot_categorized != 0).any(axis=1), :]
pivot_categorized.to_csv('documents/2019-05/Recommender System/pivot_categorized.csv')

In [11]:
pred_rating = {}

line = testSet.values[0]
myuser = line[1]
myitem = line[0]
myrate = line[2]
its_location, =np.where(reference_index == myitem)
its_location = its_location[0]

In [13]:
myitem_distances = distances[its_location:its_location+1][0]
myitem_indices = indices[its_location:its_location+1][0]
 
bi = mean_items.loc[myitem]['rating'] - mean_overall
bx = mean_users.loc[myuser]['rating'] - mean_overall
bxi = mean_overall + bi + bx

print(its_location)
print(myitem_distances)
print(myitem_indices)

print("____________________________________")


498
[0.         0.65627379 0.67031249 0.80057925]
[498 768 146 812]
____________________________________


In [21]:
pred_rating = {}

for line in testSet.values[1:20]:
    myuser = line[1]
    myitem = line[0]
    myrate = line[2]
# myuser = 'AMCAID3LTHKEC'   #testSet[1:4]['reviewerID'].iat[0]
# myitem = 'B00007LVCN'  #testSet[5:6]['asin'].iat[0]

    its_location, =np.where(reference_index == myitem)
    its_location = its_location[0]

    myitem_distances = distances[its_location:its_location+1][0]
    myitem_indices = indices[its_location:its_location+1][0]
    
    bi = mean_items.loc[myitem]['rating'] - mean_overall
    bx = mean_users.loc[myuser]['rating'] - mean_overall
    bxi = mean_overall + bi + bx
    
    
    item_occurences_in_train = []
    for i in range(4):
        item_occurence = trainSet.loc[(trainSet['itemID'] == reference_index[myitem_indices[i]]) & (trainSet['userID'] == myuser)]
        bi_n = mean_items.loc[reference_index[myitem_indices[i]]]['rating'] - mean_overall
        bix_n = mean_overall + bi_n + bx
        item_occurences_in_train.append((item_occurence, myitem_distances[i], bix_n))
    sim_rating = 0

    for line in item_occurences_in_train:
        if not line[0].empty:
            sim_rating = sim_rating + (line[0]['rating'].values-line[2])*(line[1])
            sim_rating=sim_rating[0]

    pred = bxi + sim_rating
    pred_rating[(myitem, myuser, myrate)] = pred

In [22]:
y_true = []
y_pred = []
for key in pred_rating:
    y_true.append(key[2])
    y_pred.append(pred_rating[key])

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred, multioutput='uniform_average')
print(mse)

1908.469035887094


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [ ]:
model = gl.factorization_recommender.create(sf_train, target="rating")
model.predict(testSet)